<a href="https://colab.research.google.com/github/emmangoo/gobi/blob/main/20251104_single_cell_analysis_course_exercises_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy scanpy leidenalg scvelo cellrank

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 693.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 10.8 MB/s eta 0:00:00
   

# Single-cell RNAseq data analysis project (42pts)

The ability to profile mRNA molecules in a single cell via single-cell RNA-sequencing (scRNA-seq) has revolutionized biology. First published in [2009](https://www.nature.com/articles/nmeth.1315), scRNA-seq has been around for the last 15 years in which it has been used to, e.i. characterize how [organs develop in human fetuses](https://www.science.org/lookup/doi/10.1126/science.aba7721), how whole [organisms develop from a single cell](https://pubmed.ncbi.nlm.nih.gov/29674432/), and how [pancreatic beta cells dedifferentiate in type 2 diabetes](https://www.nature.com/articles/s42255-020-0171-3), amongst many other discoveries. Today single-cell profiling methods have been named Nature Methods method of the year already 3 times in this time (e.i. long-read sequencing, spatial transcriptomics). In this worksheet we will guide you through a simple scRNA-seq analysis pipeline. Each one of the discoveries above used extensions of the basic principles in this worksheet.

In this exercise, we will leverage the python package `scanpy`. If you are not familiar with this package, please check out its [documentation](https://scanpy.readthedocs.io/en/stable/).

The tasks will guide you through Quality Control, Normalization, Feature Selection, Clustering, Annotation and Differential Expression Analysis. More details and background information about the tasks can be found in the [sc-best-practices book](https://www.sc-best-practices.org/preamble.html).



**To get started:** Copy the raw data set from the lecture gDrive folder to your Google drive.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc

To load the data we need to mount our google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In this exercise, we will use a dataset describing the endocrine development in the pancreas from [Bastidas-Ponce et al. (2018)](https://journals.biologists.com/dev/article/146/12/dev173849/19483/Comprehensive-single-cell-mRNA-profiling-reveals-a).

**Adjusting the path:** Adjust the path below according to where you saved it on your Google drive.

In [ ]:
# Modify path according to your file location
# adata = sc.read_h5ad('drive/MyDrive/PhD/2023_gobi/endocrinogenesis_day15_gobi.h5ad')
adata = sc.read_h5ad('drive/MyDrive/endocrinogenesis_day15_gobi.h5ad')

 With scanpy we load an annotated data matrix. This matrix has the following structure. Throught the exercise all analysis results will be stored in the annotated data matrix. If you want to read more about this data format, check [AnnData documentation](https://anndata.readthedocs.io/en/latest/).


 <img src="https://falexwolf.de/img/scanpy/anndata.svg" width="400">

 The main component of an AnnData object is the data matrix `X` of shape `#observations x #variables`, so `number of cells x number of genes`. Gene names and additional information on gene level are stored in `var`. Cell level, e.g. cell type annotation, is stored in `obs`. Unstructured annotations of the dataset of any form are stored in `uns`.

In [ ]:
adata

AnnData object with n_obs × n_vars = 3696 × 27998
    obs: 'clusters'

The dataset consists of 3,696 cells across 27,998 genes.

## 1.1. Preprocessing and Quality Control (6pts)

Next we will look into Preprocessing and Quality Control, read along in the [Chapter 6](https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html) of the sc-best-practices book.

**Task:** Show the genes that yield the highest fraction of counts in each single cell, across all cells (hint: `sc.pl.highest_expr_genes`). What type of genes do you mainly see? You can look up gene names on [genecards.org](https://www.genecards.org/)



In [ ]:
# ToDo: 1.1.1 Add code (1pt - correct function)


Data quality control (QC) can be split into cell QC and gene QC. Typical quality measures for assessing the quality of a cell include the number of molecule counts (UMIs), the number of expressed genes, and the fraction of counts that are mitochondrial. A high fraction of mitochondrial reads being picked up can indicate cell stress, as there is a low proportion of nuclear mRNA in the cell. It should be noted that high mitochondrial RNA fractions can also be biological signals indicating elevated respiration.

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='pct_counts_mt')

On this plot, the x-axis indicates how many transcript counts are assigned to a single cell. The y-axis indicates how many genes in an observed cell have at least one transcript count. The dots are the single cells. The color scheme is according to the percentage of mitochondrial transcript counts per individual cell's transcript counts.

**1.1.2. ToDo: answer question** (2pt)

**Question:**
- a) which cells might not meet our quality criteria, according to earlier descriptions? Please describe the section on the scatterplot (e.g. bottom right corner and blue dots) where these cells appear
- b) what is happening to them biologically?

**Answer:**
- a)
- b)






**1.1.3. Task: (2pts)** Plot the distribution of `total_counts` by using `sns.histplot`.
- a) What discrete distribution could describe this data?
- b) Filter the data for cells that have less than 4% mitochondrial counts. Consider to have a look at `adata.obs['pct_counts_mt']` for this
- c) Filter the data for cells that have at least 1200 detected genes (see `sc.pp.filter_cells`

In [ ]:
# ToDo: Add code


We also filter genes: here, we remove genes are filtered if they are not detected in at least 20 cells.

In [ ]:
# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print(f'Number of genes after gene filter: {adata.n_vars}')

**1.1.4. ToDo** (1pt)

**Question:** Why are we filtering out genes that are not detected?

**Answer:** ...
-

## 1.2. Normalization (4pts)

*See [Chapter 7](https://www.sc-best-practices.org/preprocessing_visualization/normalization.html) of sc-best-practices book.*

Up to this point the data is only available as a count matrix. Counts are representative of molecules that were captured in the scRNA-seq experiment. As not all mRNA molecules in a cell are captured, there is a variability in the total number of counts detected between cells that results from both the number of molecules that were in the cells, and the sampling. As we cannot assume that all cells contain an equal number of molecules (cell sizes can differ substantially), we have to estimate the number of molecules that were initially in the cells. In fact, we don't estimate the exact number of molecules, but instead estimate cell-specific factors that should be proportional to the true number of molecules. These are called size factors. Normalized expression values are calculated by dividing the measured counts by the size factor for the cell, so by $x_{gc} = n_{gc} \cdot \xi_{gc}$ with $x_{gc}$ being the normalized expression values, $n_{gc}$ the UM counts and $\xi_{gc}$ the size factor per gene $g$ in cell $c$.

One basic preprocessing normalization technique assumes that all size factors are equal (library size normalization to **counts per million (CPM)**, so $x_{gc} = n_{gc} \cdot \frac{C}{\sum_g n_{gc}}$ with $C$ a constant at a factor of 10. (see scanpy function `sc.pp.normalize_total`)


After normalization, data matrices are typically $\log(x+1)$‐transformed (see scanpy function `sc.pp.log1p`), which is performed to normalize the data distributions. The offset of 1 ensures that zero counts map to zeros.



**1.2.1 Task: (2pt)**

Perform CPM normalization on your dataset and subsequently $\log(x+1)$‐transform your data (see `sc.pp.normalize_total` and `sc.pp.log1p`). Plot the normalized data.

In [ ]:
# Prior to normalization, we save the raw count matrix in a new Anndata object
adata_raw = adata.copy()

In [ ]:
# ToDo: Add code


**1.2.2. ToDo (1pt)**

**Question:**

CPM Normalization gives you a relative expression value for each gene within a cell. Some genes scale with cell size, while others don't. How would normalization affect these genes differently?

**Answer:**






**1.2.3. ToDo (1pt)**

**Question**:

Which important effects has $\log(x+1)$ transformation with respect to the distances between expression changes?

**Answer:**



## 1.3 Feature Selection (4pts)

*See [Chapter 8](https://www.sc-best-practices.org/preprocessing_visualization/feature_selection.html) of the sc-best-practices book*

Here we will use a dispersion-based method to extract HVGs. This method first calculates the mean and a dispersion measure (variance/mean) for each gene in the dataset across all cells. The genes are then places into 20 bins based on their average expression. For each bin, we identify those genes whose expression values are highly variable compared to the other genes in the same bin by z-normalizing the dispersion measure (see scanpy function `sc.pp.highly_variable_genes`). scanpy stores the normalized dispersion values (the final statistics indicating the gene variability) in `adata.var["dispersions_norm"]`

 <img src="https://drive.google.com/uc?export=view&id=1P249LuuviSyiK3ELQzKjUrojD9DNT7iJ" width="500">



**1.3.1 ToDo (1pt)**

**Question** Why do we perform feature selection?

**Answer:**


**1.3.2 Task (3pt)**: Extract the top 2,000 genes of your dataset with a dispersion-based method (see `sc.pp.highly_variable_genes`). Name the 5 highest HVGs and 5 lowest HVGs.

In [ ]:
# ToDo: Add code


## 1.4 PCA (5pts)

*See [Chapter 9](https://www.sc-best-practices.org/preprocessing_visualization/dimensionality_reduction.html) in sc-best-practice book.*

Next, we run a Principal Component Analysis (PCA) on the dataset. PCA is used to emphasize variation as well as similarity, and to bring out strong patterns in a dataset. Overall, it also serves as a method for dimensionality reduction.

The figure below shows the concepts of PCA in a nutshell.

 <img src="https://drive.google.com/uc?export=view&id=1POJBu96zemFIRMtp1pMF9wZ3FsG6WzPs" width="800">

**1.4.1 Question: (1pt)** Why do we do dimensionality reduction?

**Answer:** ...

**1.4.2 Task (2pts)**: Perform a PCA on the dataset and plot the results (see `sc.tl.pca`)

In [ ]:
# ToDo: Add code



**1.4.3 Task (2pts)**: How many principal components should we consider if we want to keep the minimum number of principal components explaining at least 40% of the variance? (hint: `sc.pl.pca_variance_ratio`) ? What are the benefits and downsides of using more or less PC's for further analyses?

In [ ]:
# ToDo: Add code


**Answer:** ...

Often, we select as many PCs as needed to explain up to 80% of the variance in the dataset; here we continue with 50 PCs. These have been stored in `adata.varmp["PC"]` by `sc.tl.pca`. A representation of our data in this space has been stored in `adata.obsm["X_PCA"]`. This is automatically chosen for computations by our next algorithm for neighbor graph computation.

## 1.5 Clustering (9pts)

*See [Chapter 10](https://www.sc-best-practices.org/cellular_structure/clustering.html) in sc-best-practices book.*

Clustering single-cell data makes use of a k-nearest neighbourhood graph representation of the cells (see scanpy function `sc.pp.neighbors`). Typically, we use the `leiden` algorithm which optimizes graph modularity, a cost function that defines how connected groups of nodes in the knn graph are (`sc.tl.leiden`). Modularity, $Q$, is defined as:

$Q = \frac{1}{2m}\sum_{i,j}[A_{ij} - \lambda \frac{k_i k_j}{2m}]\delta(\sigma_i, \sigma_j)$.

Here, $m$ is the number of edges in the graph, $A_{ij}$ is the adjacency matrix of the graph, $k_i = \sum_j A_{ij}$ is the degree of node $i$, $\sigma_i$ indicates the community assignment of node $i$, and $\delta(\sigma_i, \sigma_j)$ is a Kronecker-delta function that is 1 if $\sigma_i = \sigma_j$. $\lambda$ is a variable that is assigned by the user, called the resolution parameter.

**1.5.1 Question: (1pt)**
What does a single-cell knn graph represent?

**Answer:**


**1.5.2 Question: (1pt)**
How does the resolution parameter control the clustering resolution?

**Answer**:

**1.5.3 Question: (1pt)** How does the effect of the resolution parameter differ between sparse and dense regions of the single-cell knn graph?

**Answer**


**1.5.4 Task: (3pts)** Set up a single-cell knn-graph of the pancreas data and cluster these data as you see fit. Present your clustering results as UMAP plots coloured by cluster assignment in the final report. (see e.g. `sc.tl.umap`, and `sc.pl.umap`)

In [ ]:
# For some reason, there is a numpy error while running `sc.pp.neighbors`
!pip install -U numpy

In [ ]:
# TODO: Add code


**1.5.5 Question: (3pts)**

Try a few different clustering resolutions. In some cases you will see that a cluster can be spread out across multiple places on the UMAP plot, with cells from other clusters in between. Why might this be the case? (Hint: you can explore the clustering with different resolution parameters using `sc.tl.leiden` and its `resolution` parameter)

In [ ]:
# TODO: add code

**Solution**
We see the example of a spread out cluster at resolution 0.25 and 0.5 (green cluster 2 in both cases). It can happen for 2 reasons:
1. Cells of this cluster are somehow similar, but shape of the data can't be projected at 2D plane without rupturing this cluster;
2. There are subpopulations in this cluster, but low resolution of clustering doesn't allow to split them into separate clusters. This is supported by the fact that at resolution 1 there are several clusters for these regions (11, 6, and 5).

## 1.6 Marker genes & Cell type annotation (9pts)

To interpret the clusters, we need to identify the cell types that these clusters represent. This is typically done by finding features (genes) that characterize a cluster from the data, and then comparing these features against other published studies. Someone has usually described the cell type you are looking at before. Here, we are facilitating this process by already giving you a list of cell-type markers that you can use to identify the cells.

Data-driven markers can be detected by statistical tests that assess whether a gene is more highly expressed in one cluster vs the rest. Typically we use simple tests such as a t-test or a Wilcoxon rank sum test here. These tests are implemented in scanpy under `sc.tl.rank_genes_groups`. We can assess whether the gene markers we find are meaningful by comparing their expression across clusters for example in a dotplot (`sc.pl.dotplot`) or by plotting their expression in a UMAP visualization (`sc.tl.umap` and `sc.pl.umap`).

In [ ]:
ct_markers = {
    'Beta': ['Ins1', 'Ins2'],
    'Alpha': ['Gcg', 'Arx'],
    'Epsilon': ['Ghrl', 'Irs4'],
    'Delta': ['Sst', 'Hhex'],
    'Ductal': ['Sox9', 'Anxa2'],
    'Endocrine progenitor' :['Bicc1', 'Fev'],
    'Ngn3 high EP': ['Neurog3', 'Ppp1r14a'],
    'Ngn3 low EP': ['Actg1', 'Spp1']
}

In [ ]:
import itertools
all_markers = list(itertools.chain(ct_markers.values()))
flat_markers = list(itertools.chain(*all_markers)) + ['leiden']
sc.pl.umap(adata, color=flat_markers)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.pl.dotplot(adata, ct_markers, groupby="leiden")

**1.6.1 Task: (3pts)**

Identify the cell types in this pancreas dataset based on the marker genes given in the dictionary `ct_markers` above. Present these results as a UMAP (`sc.tl.umap` and `sc.pl.umap`) coloured by cell type annotation. Note, you may have to go back to the subclustering step to improve your clustering (hint: setting the resolution parameter to 0.5 or 1 works fairly well for this dataset.)




In [ ]:
# Todo: the 0, 1, 2, ... are the leiden cluster labels for the individual cells in adata.obs["leiden"]
# Populate this dictionary for the leiden clusters you have (likely around 15), with the cell type from ct_markers. Multiple clusters could actually have the same cell type labels!
annotation = {
    "0": "XXXX",
    "1": "XXXX",
    "2": "XXXX",
}

In [ ]:
adata.obs["manual_celltype_annotation"] = adata.obs["leiden"].map(annotation)

In [ ]:
sc.pl.umap(adata, color=["manual_celltype_annotation", "leiden"], ncols=2)

**1.6.2 Task: (1pt)**

Find additional marker genes for each of the cell types beyond the markers that are provided. Can you find markers that are cell-type specific and some that are present across cell types?

In [ ]:
# Add code

**1.6.3 Task: (2pt)**

Can you find any other cellular substructures in the data that are not described well by only the cell type markers given? Try to cluster these and give marker genes for the clusters.

Note: check out the `restrict_to` parameter in the `sc.tl.leiden` function to perform a subclustering.

In [ ]:
# Add code


In [ ]:
# sc.pl.umap(adata, color=<add the new subtypes>) # uncomment

In [ ]:
# sc.tl.rank_genes_groups(adata, <add the new subtypes>, method='t-test') # uncomment
# sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False) #uncomment

**1.6.4 Task: (2pts)**

Remove Alpha cells from the dataset. And redo the marker gene detection for beta cells. Why do the marker genes for beta cells change?


In [ ]:
# TODO: add code

**Answer**:

**1.6.5 Question: (1pt)**

What issues can this change in marker genes cause for defining cell type markers for particular cell types? Discuss how this issue might be avoided.

**Answer:**

## 1.7 Attempting annotation with default leiden clustering (3pts)

How many populations do you see and would be able to call from this following UMAP?

In [ ]:
sc.pl.umap(adata, color='leiden')

Obviously the answer would be >10, however, this is a pre-defined clustering algorithm result, with a default parameter and a deterministic output. Let's inspect a bit more by visualizing the gene features that are most associated with the different subgroups (i.e. rank_genes_groups).

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, groupby="leiden", figsize=[18, 4], cmap='RdBu_r', n_genes=5, standard_scale='var')

Here pay attention that, based on hierarchical clustering (shown as tree on the right of this dotplot) one could label 3 major groups as well.

**1.7.1 Task: (1pt)**
Adjust the cluster numbers below (in this text cell and the code) so they match your three major groups:

In [ ]:
# Todo: extend the dictionaries starting with 3, 8, and 15 with leiden cluster numbers. Maybe you need to remove these numbers 3, 8, 15
adata.obs['cell_type_guess'] = np.where(adata.obs['leiden'].astype(int).isin({3, }), 'A', 'N/A')
adata.obs['cell_type_guess'] = np.where(adata.obs['leiden'].astype(int).isin({8, }), 'B', adata.obs['cell_type_guess'])
adata.obs['cell_type_guess'] = np.where(adata.obs['leiden'].astype(int).isin({15, }), 'C', adata.obs['cell_type_guess'])

In [ ]:
sc.pl.umap(adata, color='cell_type_guess')

This visualization would be very similar to the one obtained when executing the leiden clustering with a lower resolution parameter.

**1.7.2 Task: (1pt)**

Compare this visualization with a coarser leiden clustering.

In [ ]:
# TODO: add code

**1.7.3 Task: (1pt)**

The red/blue dotplot above might have repeated genes. This is cause by the top-N genes reported for one group of cells partly overlapping with the top-N genes of another one.
i) do you detect repeated genes the dotplot (there is at least one).

**Solution**

## 1.8 Score cell cycle genes (1pt)

In general terms, cell cycle processes are a part of scRNA-seq datasets, and its has to be taken into account, scored and labeled.

For this, as a general processing step, cell cycle genes known to vary across different cell cycle stages, are scored per cell. Then, a very simple gating algorithm is used to define whether some cells are more likely S in phase, G2M, or *G1*

In [ ]:
!wget https://raw.githubusercontent.com/theislab/scanpy_usage/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt

In [ ]:
cell_cycle_genes = [x.strip() for x in open('regev_lab_cell_cycle_genes.txt')]
cell_cycle_genes = [x.capitalize() for x in cell_cycle_genes]

In [ ]:
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
# generate a coarser leiden annotation
sc.tl.leiden(adata, resolution=.25, key_added='leiden.coarse')

In [ ]:
sc.pl.umap(adata, color=['leiden.coarse', 'S_score', 'G2M_score', 'phase'], ncols=2)

Using the cc_score, one neat observation is that some of the coarse leiden clusters are associated with cycling cells.

**1.8.1 Task (1pt)**

Identify the coarse clusters that refer to cycling cells and re-label them. To do so, modify the XX / 'XX' accordingly.

In [ ]:
# Modify XX according to clusters corresponding to cycling cells
# adata.obs['cell_type_guess'] = np.where(adata.obs['leiden.coarse'].astype(int).isin({XX, XX}), 'cycling cells', adata.obs['cell_type_guess'])

In [ ]:
sc.pl.umap(adata, color='cell_type_guess')

Combining both hierarchical clustering, leiden and cycling cell scores into one label could be useful to keep making decisions

In [ ]:
adata.obs['cell_type_guess'] = adata.obs['cell_type_guess'].astype(str) + '.' + adata.obs['leiden.coarse'].astype(str)
adata.obs['cell_type_guess'] = np.where(adata.obs['cell_type_guess'].str.startswith('cycling'), 'cycling cells', adata.obs['cell_type_guess'])

In [ ]:
sc.pl.umap(adata, color='cell_type_guess')

Let's see rank genes groups one more time

In [ ]:
sc.tl.rank_genes_groups(adata, 'cell_type_guess')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, figsize=[10, 2], cmap='RdBu_r', n_genes=5, standard_scale='var')

Now we have a decently interpretable matrix, but this is still not finished. How can we possibly obtain a name for these cells?

## 1.9 Guessing cell clusters through an iterative process (1pt)
Before having a follow-up meeting with your collaborators, it would be reasonable to inspect a bit on the Biology of your system by reading up in databases where knowledge about your cells and the markers genes you're observing have been described before.
- e.g.1. Based on GeneCards info, Fev is as an [transcription factor of endocrine cells](https://www.genecards.org/cgi-bin/carddisp.pl?gene=FEV&keywords=Fev#expression). This statement alone could be useful to recommend a cluster YYYY as an endocrine cell population.
- e.g.2. [Sox4 interacts with Neurog3 during Beta cell development](https://pubmed.ncbi.nlm.nih.gov/25652387/) Probably a cluster ZZZZ elevated in both should be updated to a Ngn3+, transitioning cluster?


In [ ]:
### TODO: replace YYYY with a cluster that has elevated Fev
#adata.obs['cell_type_guess'] = adata.obs['cell_type_guess'].str.replace(YYYY, 'Fev2')
### TODO: replace ZZZZ with a cluster that has elevated Sox4 and Neurog3
#adata.obs['cell_type_guess'] = adata.obs['cell_type_guess'].str.replace(ZZZZ, 'Ngn3+')

In [ ]:
sc.pl.umap(adata, color='cell_type_guess')

**1.9.1 Task: (1pt)**

Try to provide labels for at least one the of the remaining, unlabeled groups.

*Tip*: Endocrine-progenitor cells differentiate into specific population of pancreatic cells, among others, Alpha/Beta/Gamma/Delta/Epsilon (greek letters).
[See Figure 3-4 of this paper for some additional gene markers](https://pubmed.ncbi.nlm.nih.gov/28951822)

In [ ]:
# Add code

**Before next section**

Before starting the DE analysis, we will update the labels of our current object with the ones that have been already previously processed by the analysis of this dataset. Those will be overwritten in our current object.

[**Reference for annotation**](https://scvelo.readthedocs.io/scvelo.datasets.pancreas/)

In [ ]:
adata.obs['cell_type_ann'] = adata.obs['clusters']
# for learning purposes, please do not uncomment and print this visualization step until you're advanced in the cell annotation tasks above.
# sc.pl.umap(adata, color='cell_type_ann')

# 2. DE analysis (5pts)

Differential expression analysis is a group of statistical tests that are used to establish whether there a exists a significant variation across a set of tested conditions for each gene. In its easiset form, this test can test for the difference between two distinct groups: This scenario can be handled with for example (Welch's) T-test, rank sum tests or Wald and likelihood ratio tests (LRT). Wald tests and LRT allow for more adaptive assumptions on the noise model and can therefore be more statistically correct. Moreover, they also allow the testing of more complex effect, e.g. for the variation across many groups (a single p-value for: Is there any difference between four conditions?) or across continuous covariates (a single covariate for: Is a gene expression trajectory in time non-constant?).

**2.1 Task (2pts)**

Using Scanpy's 'rank_genes_groups' function calculate a t-test and a Wilcoxon rank sum test between Alpha and Beta cells. You can use your own annotation if it fits or the 'cell_type_ann' annotation.

In [ ]:
# Add code


**Question**

What is the difference in assumptions between these tests?


**Answer**


**2.2 Task (2pts)**

Now plot the top 30 differentially expressed genes for both of the tests. Do they overlap? What are the top 3 differentially expressed genes?



In [ ]:
# Add code


**2.3 Question (1pt)**

We calculated the differentially expressed genes using the normalized counts. Should differentially expressed genes be calculated using the unnormalized or normalized data? Explain your reasoning.

# 3. Trajectory *Inference* (5pts)

Trajectory inference approaches analyze genome-wide omics data from thousands of single cells and computationally infer the order of these cells along developmental trajectories.

Many methods for the inference of trajectories exist based on statistical approaches, RNA velocity or optimal transport (see advanced topics below). Here, we introduce [diffusion pseudotime](https://www.nature.com/articles/nmeth.3971), an efficient way of robustly estimating the ordering according to diffusion pseudotime, which measures the transitions between cells using diffusion-like random walks.

Convert UMAP indices to arrays.

In [ ]:
umap_0 = [term[0] for term in adata.obsm['X_umap']]
umap_1 = [term[1] for term in adata.obsm['X_umap']]

Set root cell to the cell with the smallest value in the first UMAP component and compute DPT.

**3.1 Question (1pt)**

Why are we selecting the cell with the smallest value in the first UMAP component as the root cell?

**Answer**:

In [ ]:
adata.uns['iroot'] = np.flatnonzero(umap_0== max(umap_0))[0]
sc.tl.dpt(adata = adata)

In [ ]:
sc.pl.umap(adata, color=['dpt_pseudotime'])

**3.2 Task (2pts)**

Plot the diffusion pseudotime and the cell types based on the component combinations 1,2 and 2,3 in a UMAP.

In [ ]:
# TODO:
# sc.pl.diffmap(adata, color="manual_celltype_annotation", components=[<fill in this list with the two pairs of components>])

Single-cell RNA-seq quantifies biological heterogeneity across both discrete cell types and continuous cell transitions. [Partition-based graph abstraction](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1663-x) (PAGA) provides an interpretable graph-like map of the arising data manifold, based on estimating connectivity of manifold partitions. PAGA maps preserve the global topology of data, allow analyzing data at different resolutions, and result in much higher computational efficiency of the typical exploratory data analysis workflow.
In simpler words, PAGA provides an estimation of the connectivity of clusters while maintaining the cluster distribution in space.


**3.3 Task (2pts)**

Calculate and plot a PAGA graph with Scanpy using the 'cell_type' annotation. Which cells are connected? What does the thickness of the arrows mean? What does the result tell you about the biology? Attempt an interpretation.

**Answer**


In [ ]:
sc.tl.paga(adata, groups="manual_celltype_annotation")

In [ ]:
sc.pl.paga(adata)

# 4. Advanced Topics (**Optional**)

The advanced topics are not part of the core exercise sheet and are **entirely optional**. However, they do award bonus points which can offset point losses in the earlier exercises.
By design they do not necessarily introduce all concepts in depth. However, references to core papers and documentation are provided to encourage exploration.

## 4.1 RNA Velocity (8pts)

Here you will learn the basics of [RNA velocity](https://www.nature.com/articles/s41586-018-0414-6) analysis. Check [chapter 14](https://www.sc-best-practices.org/trajectories/rna_velocity.html) of the best practices book as well.

RNA abundance is a powerful indicator of the state of individual cells. Single-cell RNA sequencing can reveal RNA abundance with high quantitative accuracy, sensitivity and throughput. However, this approach captures only a static snapshot at a point in time, posing a challenge for the analysis of time-resolved phenomena such as embryogenesis or tissue regeneration. Here we show that RNA velocity—the time derivative of the gene expression state—can be directly estimated by distinguishing between unspliced and spliced mRNAs in common single-cell RNA sequencing protocols. RNA velocity is a high-dimensional vector that predicts the future state of individual cells on a timescale of hours.

For illustration, it is applied to endocrine development in the pancreas, with lineage commitment to four major fates: α, β, δ and ε-cells. <br/>
See [here](https://scvelo.readthedocs.io/scvelo.datasets.pancreas.html) for more details.

To analyze the RNA velocity of our Pancreas dataset we will use [scVelo](https://www.nature.com/articles/s41587-020-0591-3).


In [ ]:
!pip install git+https://github.com/theislab/scvelo

In [ ]:
!pip install -U numpy

In [ ]:
import numpy
numpy.__version__

In [ ]:
import scvelo as scv

In [ ]:
scv.settings.verbosity = 3
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')  # for beautified visualization

The analysis is based on the in-built [pancreas data](https://scvelo.readthedocs.io/scvelo.datasets.pancreas.html) as used above. Let's get a clean and already annotated copy.

In [ ]:
adata = scv.datasets.pancreas()
adata

The proportion of unsplied and spliced counts is the basis of RNA velocity. Let's have a look:

In [ ]:
scv.pl.proportions(adata)

Here, the proportions of spliced/unspliced counts are displayed. Depending on the protocol used (Drop-Seq, Smart-Seq), we typically have between 10%-25% of unspliced molecules containing intronic sequences. We also advice the examination of the variations on cluster level to verify consistency in splicing efficiency. Here, we find variations as expected, with slightly lower unspliced proportions at cycling ductal cells, then higher proportion at cell fate commitment in Ngn3-high and Pre-endocrine cells where many genes start to be transcribed.

Preprocessing requisites consist of **gene selection** by detection (with a minimum number of counts) and high variability (dispersion), **normalizing** every cell by its total size and **logarithmizing** X. Filtering and normalization is applied in the same vein to spliced/unspliced counts and X. Logarithmizing is only applied to X. If X is already preprocessed from former analysis, it will not be touched.

All of this is summarized in a single function `scv.pp.filter_and_normalize`, which essentially runs the following:

```
scv.pp.filter_genes(adata, min_shared_counts=20)
scv.pp.normalize_per_cell(adata)
scv.pp.filter_genes_dispersion(adata, n_top_genes=2000)
scv.pp.log1p(adata)
```

Further, we need the first and second order moments (means and uncentered variances) computed among nearest neighbors in PCA space, summarized in `scv.pp.moments`, which internally computes `scv.pp.pca` and `scv.pp.neighbors`.
First order is needed for deterministic velocity estimation, while stochastic estimation also requires second order moments.

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)


Velocities are vectors in gene expression space and represent the direction and speed of movement of the individual cells. The velocities are obtained by modeling transcriptional dynamics of splicing kinetics, either stochastically (default) or deterministically (by setting ``mode='deterministic'``). For each gene, a steady-state-ratio of pre-mature (unspliced) and mature (spliced) mRNA counts is fitted, which constitutes a constant transcriptional state. Velocities are then obtained as residuals from this ratio. Positive velocity indicates that a gene is up-regulated, which occurs for cells that show higher abundance of unspliced mRNA for that gene than expected in steady state. Conversely, negative velocity indicates that a gene is down-regulated.

The solution to the full dynamical model is obtained by setting ``mode='dynamical'``, which requires to run
``scv.tl.recover_dynamics(adata)`` beforehand. We skip the dynamic model in this exercise sheet.

**4.1.1 Task (1pt)**

Compute the velocities with scVelo. Consider to use `scv.tl.velocity`.

In [ ]:
# TODO: add code.

computing velocities


/usr/local/lib/python3.10/dist-packages/scvelo/tools/optimization.py:184: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gamma[i] = np.linalg.pinv(A.T.dot(A)).dot(A.T.dot(y[:, i]))


    finished (0:00:01) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


The computed velocities are stored in ``adata.layers`` just like the count matrices.

The combination of velocities across genes can then be used to estimate the future state of an individual cell. In order to project the velocities into a lower-dimensional embedding, transition probabilities of cell-to-cell transitions are estimated. That is, for each velocity vector we find the likely cell transitions that are accordance with that direction. The transition probabilities are computed using cosine correlation between the potential cell-to-cell transitions and the velocity vector, and are stored in a matrix denoted as velocity graph. The resulting velocity graph has dimension $n_{obs} \times n_{obs}$ and summarizes the possible cell state changes that are well explained through the velocity vectors (for runtime speedup it can also be computed on reduced PCA space by setting `approx=True`).

**4.1.2 Task (1pt)**

Compute the velocity graph with scVelo. Consider to use `scv.tl.velocity_graph`

In [ ]:
# Compute velocity graph


For a variety of applications, the velocity graph can be converted to a transition matrix by applying a Gaussian kernel to transform the cosine correlations into actual transition probabilities. You can access the Markov transition matrix via `scv.utils.get_transition_matrix`.

As mentioned, it is internally used to project the velocities into a low-dimensional embedding by applying the mean transition with respect to the transition probabilities, obtained with `scv.tl.velocity_embedding`. Further, we can trace cells along the Markov chain to their origins and potential fates, thereby getting root cells and end points within a trajectory, obtained via `scv.tl.terminal_states`.

Finally, the velocities are projected onto any embedding, specified by `basis`, and visualized in one of these ways:
- on cellular level with `scv.pl.velocity_embedding`,
- as gridlines with `scv.pl.velocity_embedding_grid`,
- or as streamlines with `scv.pl.velocity_embedding_stream`.

Note, that the data has an already pre-computed UMAP embedding, and annotated clusters. When applying to your own data, these can be obtained with `scv.tl.umap` and `scv.tl.louvain`. For more details, see the [scanpy tutorial](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html). Further, all plotting functions are defaulted to using `basis='umap'` and `color='clusters'`, which you can set accordingly.

**4.1.3 Task (1pt)**

Plot the velocity embedding as a stream and examine the result. Which proposed transitions can you see? Is the result biologically sound? Consider to use `scv.pl.velocity_embedding_stream`.

In [ ]:
# Plot velocity embedding as stream

The most fine-grained resolution of the velocity vector field we get at single-cell level, with each arrow showing the direction and speed of movement of an individual cell.

**4.1.4 Task (2pts)**

Plot the velocity embedding and examine the Ngn3-cells, the α-cells and transient β-cells. What do you see? Consider to use `scv.pl.velocity_embedding`.

In [ ]:
# Plot velocity embedding


This is perhaps the most important part as we advise not to limit biological conclusions to the projected velocities, but to examine individual gene dynamics via phase portraits to understand how inferred directions are supported by particular genes.

See the gif [here](https://user-images.githubusercontent.com/31883718/80227452-eb822480-864d-11ea-9399-56886c5e2785.gif) to get an idea of how to interpret a spliced vs. unspliced phase portrait. Gene activity is orchestrated by transcriptional regulation. Transcriptional induction for a particular gene results in an increase of (newly transcribed) precursor unspliced mRNAs while, conversely, repression or absence of transcription results in a decrease of unspliced mRNAs. Spliced mRNAs is produced from unspliced mRNA and follows the same trend with a time lag. Time is a hidden/latent variable. Thus, the dynamics needs to be inferred from what is actually measured: spliced and unspliced mRNAs as displayed in the phase portrait.

Now, let us examine the phase portraits of some marker genes, visualized with <br/>
`scv.pl.velocity(adata, gene_names)` or `scv.pl.scatter(adata, gene_names)`.

**4.1.5 Task (1pt)**

Plot the phase portraits for the genes 'Cpe', 'Gnao1', 'Ins2' and 'Adk' with scVelo. Consider to use `scv.pl.scatter`.

In [ ]:
# Plot phase portraits


The black line corresponds to the estimated 'steady-state' ratio, i.e. the ratio of unspliced to spliced mRNA abundance which is in a constant transcriptional state. RNA velocity for a particular gene is determined as the residual, i.e. how much an observation deviates from that steady-state line. Positive velocity indicates that a gene is up-regulated, which occurs for cells that show higher abundance of unspliced mRNA for that gene than expected in steady state. Conversely, negative velocity indicates that a gene is down-regulated.

For instance *Cpe* explains the directionality in the up-regulated Ngn3 (yellow) to Pre-endocrine (orange) to β-cells (green), while *Adk*  explains the directionality in the down-regulated Ductal (dark green) to Ngn3 (yellow) to the remaining endocrine cells.

**4.1.5 Task (2pts)**

Plot a scatterplot of the gene 'Cpe' colored by 'clusters' and 'velocity'. Add an outline for 'Ngn3 high EP, Pre-endocrine, Beta'. What do you see? Consider to use `scv.pl.scatter`.


In [ ]:
# Plot scatterplot


## 4.2 Cell Fate Mapping (10pts)

Fate mapping describes the studying of embryonic origin of adult tissue, cell types and structures. Here, the "fate" of every cell is mapped against the embryo highlighting which part of the embryo will develop into which tissue. This process is also commonly known as cell lineage tracing at the single-cell level where it is also commonly applied to trace tumor development.

One of several tools to determine cell fates is **CellRank**.

This tutorial shows how to work with CellRank using the **low-level mode**. We will interact directly with CellRank's two main modules, [kernels](https://cellrank.readthedocs.io/en/latest/classes.html#kernels) and [estimators](https://cellrank.readthedocs.io/en/latest/classes.html#estimators). For more info on CellRank, see the [documentation](https://cellrank.readthedocs.io/en/latest/) or read the [preprint](https://doi.org/10.1101/2020.10.19.345983).

The data we use here comes from [Bastidas-Ponce et al. (2018)](https://dev.biologists.org/content/146/12/dev173849.abstract). The AnnData object we download has been preprocessed already and velocities have been computed using scVelo's dynamical model, see the [scVelo tutorial](https://scvelo.readthedocs.io/DynamicalModeling.html) on this for more details.

In [ ]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np

scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo')
cr.settings.verbosity = 2

First, we need to get the data. The following commands will download the `adata` object and save it under `datasets/endocrinogenesis_day15.5_preprocessed.h5ad`.

In [ ]:
adata = cr.datasets.pancreas(kind="preprocessed")
scv.utils.show_proportions(adata)
adata

As stated above, this data has been preprocessed already. We can inspect scVelo's computed velocities:

In [ ]:
scv.pl.velocity_embedding_stream(adata)

CellRank is a package for analyzing directed single cell data, whereby we mean single cell data that can be respresented via a directed graph. Most prominently, this is the case for single cell data where velocities have been computed - we can use these to direct the KNN graph. However, there are other situations in which we can inform the KNN graph of the direciton of the process, using i.e. pseudotime (see [Palantir](https://doi.org/10.1038/s41587-019-0068-4)) or information obtained via mRNA labeling with e.g. [scSLAM-seq](https://doi.org/10.1038/s41586-019-1369-y), [scEU-seq](https://doi.org/10.1126/science.aax3072) or [sci-fate](https://doi.org/10.1038/s41587-020-0480-9). Because we wanted CellRank to be widely applicable, no matter how directionality was introduced to the data, we split it up into two main modules, `kernels` and `estimators`. In short, `kernels` allow you to compute a (directed) transition matrix, whereas `estimators` allow you to analyze it.

To construct a transition matrix, CellRank offers a number of kernel classes in `cellrank.tl.kernels`. Currently implemented are the following:

 - `VelocityKernel`: compute transition matrix based on RNA velocity.
 - `ConnectivityKernel`: compute symmetric transition matrix based on transcriptomic similarity (essentially a [DPT](https://doi.org/10.1038/nmeth.3971) kernel).
 - `PalantirKernel`: mimics [Palantir](https://doi.org/10.1038/s41587-019-0068-4). Uses pseudotime to direct the KNN graph.

These kernels can be combined by simply using the `+` or `*` operator, we will demonstrate this below. To find out more, check out the [API](https://cellrank.readthedocs.io/en/latest/api.html#kernels). Note that the `kernel` classes are designed to be easy to extend to incoporate future kernels based on e.g. mRNA labeling or other sources of directionality. Let's start with the `VelocityKernel`:

In [ ]:
pip install "numpy<2.0.0"

In [ ]:
from cellrank.kernels import VelocityKernel
vk = VelocityKernel(adata)

To learn more about this object, we can print it:

In [ ]:
print(vk)

There is not very: much there yet. We can change this by computing the transition matrix.

**4.2.1 Task: (2pts)**

Compute the transition matrix and examine the velocity kernel again. Consider to use `vk.compute_transition_matrix`.

In [ ]:
# TODO: add code

In [ ]:
vk

There's a lot more info now!
Explain in a couple of words each what the mode, backward and softmax_scale properties describe.

**4.2.2 Task (1pt)**

Import the connectivity kernel form `cellrank.kernels` and use it to compute a transition matrix. Name its output `ck`.

In [ ]:
from cellrank.kernels import ConnectivityKernel

# ck = ...

**4.2.3 Task (1pt)**

Define a variable combined_kernel which combines both kernels, but weights the velocity kernel by 80% and the connectivity kernel by 20%.

In [ ]:
#combined_kernel = 0.8 * ... + 0.2 * ...
#combined_kernel

(0.8 * VelocityKernel[n=2531, model='deterministic', similarity='correlation', softmax_scale=3.974] + 0.2 * ConnectivityKernel[n=2531, dnorm=True, key='connectivities'])

Estimators take a `kernel` object and offer methods to analyze it. The main objective is to decompose the state space into a set of metastable states (also called macrostates) that represent the slow-time scale dynamics of the process. A subset of these metastable states will be the initial or terminal states of the process, the remaining states will be intermediate transient states. CellRank currently offers two estimator classes in `cellrank.tl.estimators`:

 - `CFLARE`: **C**lustering and **F**iltering **L**eft **A**nd **R**ight **E**igenvectors. Heuristic method based on the spectrum of the transition matrix.
 - `GPCCA`: **G**eneralized **P**erron **C**luster **C**luster **A**nalysis: project the Markov chain onto a small set of macrostates using a Galerkin projection which maximizes the self-transition probability for the macrostates, see [Reuter et al. (2018)](https://doi.org/10.1021/acs.jctc.8b00079).

For more information on the estimators, have a look at the [API](https://cellrank.readthedocs.io/en/latest/api.html#estimators). We will demonstrate the `GPCCA` estimator here, however, the `CFLARE` estimator has a similar set of methods (which do different things internally). Let's start by initializing a `GPCCA` object based on the `combined_kernel` we constructed above, i.e. the one that uses both RNA velocities and connectivities.

In [ ]:
from cellrank.estimators import GPCCA
g = GPCCA(combined_kernel)
print(g)

GPCCA[kernel=(0.8 * VelocityKernel[n=2531] + 0.2 * ConnectivityKernel[n=2531]), initial_states=None, terminal_states=None]


In addition to the information about the kernel it is based on, this prints out the number of states in the underlying Markov chain. GPCCA needs a real sorted [Schur decomposition](https://en.wikipedia.org/wiki/Schur_decomposition) to work with.

**4.2.4 Task (1pt)**

Compute a Schur decomposition of the transition matrix using the `g` object from above and plot the eigenvalues in complex plane. The CellRank documentation will help.

In [ ]:
# TODO: add code

To compute the Schur decomposition, there are two methods implemented

 - `method='brandts'`: use `scipy.linalg.schur` to compute a full real Schur decomposition and sort it using a python implementation of [Brandts (2002)](https://doi.org/10.1002/nla.274). Note that `scipy.linalg.schur` only supports dense matrices, so consider using this for small cell numbers (<10k).
 - `method='krylov'`: use an interative, krylov-subspace based algorightm provided in [SLEPc](https://slepc.upv.es/) to directly compute a partial, sorted, real Schur decomposition. This works with sparse matrices and will scale to extremly large cell numbers.

The real Schur decomposition for transition matrix `T` is given by `Q U Q**(-1)`, where `Q` is orthogonal and `U` is quasi-upper triangular, which means it's upper triangular except for 2x2 blocks on the diagonal. 1x1 blocks on the diagonal represent real eigenvalues, 2x2 blocks on the diagonal represent complex eigenvalues. Above, we plotted the top 20 eigenvalues of the matrix `T` to see whether there is an apparent _eigengap_. In the present case, there seems to be such a gap after the first 3 eigenvalues. We can visualize the corresponding Schur vectors in the embedding:

In [ ]:
g.plot_spectrum()

These vectors will span an invariant subspace, let's call it `X` (Schur vectors in the columns). The next step in `GPCCA` is to find a linear combination of these vectors such that the Markov chain defined on the subset of states has large selt-transition probability.

**4.2.5 Task (2pts)**

Compute and plot the macrostates using 3 states. Afterwards plot the individual states. Finally, plot only the most likely cells for each state (discrete).

In [ ]:
# Compute and plot the macrostates here.


Finally, save your results in the AnnData object using the appropriate CellRank function.

Once we found the terminal states in this dataset, the next big questions is: how likely is each cell to develop towards each of these? In CellRank, we compute this via so-called "absorption probabilities".

**4.2.6 Task (2pts)**

Compute absorption probabilities using the `g` object and visualise them in the embedding, so that we have one plot per lineage

In [ ]:
# Calculate absorption probabilites here.


In [ ]:
g.plot_fate_probabilities(same_plot=False)

Once we have set the `absorption_probabilities`, we can correlate them against all genes to find potential lineage drivers. Below, we show how to do this for just one state. Note that results are written to the `.var` attribute of `adata`:

In [ ]:
alpha_drivers = g.compute_lineage_drivers(lineages='Alpha', return_drivers=True)
alpha_drivers.sort_values(by="Alpha_corr", ascending=False)

We can look at some of the identified genes:

In [ ]:
g.plot_lineage_drivers("Alpha", n_genes=5)

To find genes which may be involved in fate choice early on, we could restrict the correlation to a subset of our clusters using the `cluster_key` and `clusters` parameters.

**4.2.7 Task (1pt)**

To bring it all together visualize the fate probabilites towards all terminal states jointly in a single plot with a circular embedding using CellRank. Consider to use `cr.pl.circular_projection`.

In [ ]:
# Plot circular embedding
